# Instalação de dependências

In [3]:
!pip install binary gensim==3.6.0

  Using cached gensim-3.6.0-cp38-cp38-linux_x86_64.whl


In [2]:
#!pip3 uninstall gensim -y
!pip3 uninstall numpy -y    
#!apt-get install python3-dev build-essential      
#!pip3 install --upgrade gensim

Found existing installation: numpy 1.19.5
Uninstalling numpy-1.19.5:
  Successfully uninstalled numpy-1.19.5


In [3]:
!pip install numpy

     |████████████████████████████████| 15.7 MB 3.8 MB/s            
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.5 which is incompatible.
tensorflow-gpu 2.6.0 requires numpy~=1.19.2, but you have numpy 1.21.5 which is incompatible.


In [1]:
!pip install gensim==3.6.0
#!pip install gensim==3.7.1
#!pip install gensim==4.0.1

  Using cached gensim-3.6.0-cp38-cp38-linux_x86_64.whl
  Attempting uninstall: gensim
    Found existing installation: gensim 4.1.2
    Uninstalling gensim-4.1.2:
      Successfully uninstalled gensim-4.1.2


# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import csv
import multiprocessing
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from time import time  # To time our operations

#import logging  # Setting up the loggings to monitor gensim
#logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

# Funções

In [2]:
import gensim
def deleteSentinel(word, count, min_count):
  if (word == 'Sentinel'):
    print(word)
    return gensim.utils.RULE_DISCARD
  else:
    return gensim.utils.RULE_DEFAULT

# Tuplas
Seção de leitura e treinamento dos dados no formado de tuplas

## Treinamento Separado

### Dados em Tuplas
Nesse modelo os dados estão no formato <centro, vizinho>

In [63]:
#Quantidade de tuplas de vizinhos nos dados do yelp (center, context) considerando d = 100m
tuples = pd.read_csv('austin-sl-tuple-n-itdl-0bin-wgt0.3.csv')
print(tuples.shape)
tuples = tuples.dropna()
print(tuples.shape)

(66091, 3)
(66091, 3)


In [64]:
##Utilizando apenas os tipos dos PoIs para criação das sentenças no Word2Vec
tuples = tuples[['centerPoI', 'contextPoI']]
tuples.head()

,centerPoI,contextPoI
0,Doctors,Pediatricians
1,Health & Medical,Pediatricians
2,Internal Medicine,Pediatricians
3,Family Practice,Pediatricians
4,Massage Therapy,Doctors


In [68]:
sentencesTuples = tuples.values.tolist()

### Treinamento Tupla
Modelo da versão que utiliza tuplas para treinamento

In [ ]:
#p2v_modeltp = Word2Vec.load('austin-sl-sentences-n-itdl-0bin-wgt0.3-wnd5-d.model')
#a = KeyedVectors.load_word2vec_format('austin-sl-sentences-n-itdl-0bin-wgt0.3-wnd5-d.model')

In [65]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

In [66]:
p2v_modeltp = Word2Vec(min_count=1,
                     #window=2,
                     sg=1, #Skip-gram
                     window=1,
                     size=70,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [69]:
t = time()
p2v_modeltp.build_vocab(sentencesTuples, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.0 mins


In [70]:
t = time()
p2v_modeltp.train(sentencesTuples, total_examples=p2v_modeltp.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.05 mins


In [ ]:
p2v_modeltp.save('yelpdecatur-sl-tuple-n-itdl-8bin-wgt0.7.model')

In [74]:
p2v_modeltp.wv.most_similar(positive=['Pizza'])

[('Pool Halls', 0.9996172189712524),
 ('Farmers Market', 0.999565601348877),
 ('Eyewear & Opticians', 0.999527096748352),
 ('Home & Rental Insurance', 0.9995150566101074),
 ('Bankruptcy Law', 0.9995096325874329),
 ('Personal Care Services', 0.9995072484016418),
 ('Guns & Ammo', 0.9995056390762329),
 ('Modern European', 0.9994992017745972),
 ('Tacos', 0.9994903206825256),
 ('Korean', 0.9994902014732361)]

In [ ]:
p2v_modeltp.wv.most_similar(positive=['Pizza'])

[('Health & Medical', 0.9993394613265991),
 ('Florists', 0.9993362426757812),
 ('Neurologist', 0.9993259906768799),
 ('Food Trucks', 0.999309241771698),
 ('Firewood', 0.9993055462837219),
 ('Pressure Washers', 0.999303936958313),
 ('Self-defense Classes', 0.9993013143539429),
 ('Carpenters', 0.9993000030517578),
 ('Wheel & Rim Repair', 0.9992949962615967),
 ('Motorsport Vehicle Repairs', 0.9992905259132385)]

In [71]:
p2v_modeltp.wv.most_similar(positive=['Coffee & Tea'])

[('Food', 0.9996383190155029),
 ('Oil Change Stations', 0.9995875954627991),
 ('Home Energy Auditors', 0.9995433688163757),
 ('Caribbean', 0.9995304346084595),
 ('Mini Golf', 0.9995277523994446),
 ('Mailbox Centers', 0.9995277523994446),
 ('Pool Halls', 0.9995244741439819),
 ('Apartments', 0.9995231032371521),
 ('Wheel & Rim Repair', 0.9995213747024536),
 ('Accessories', 0.9995169639587402)]

In [76]:
p2v_modeltp.wv.most_similar(positive=['Shopping'])

[('Audio/Visual Equipment Rental', 0.9995119571685791),
 ('French', 0.9994892477989197),
 ('Tiling', 0.9994289875030518),
 ('Pilates', 0.9994286298751831),
 ('Wineries', 0.9994147419929504),
 ('Food Trucks', 0.9994110465049744),
 ('Steakhouses', 0.9994009733200073),
 ('Musicians', 0.9993941187858582),
 ('Arts & Entertainment', 0.9993936419487),
 ("Men's Clothing", 0.9993844628334045)]

In [ ]:
p2v_modeltp.wv.similarity("Pizza", 'Bars')

0.69876593

In [ ]:
p2v_modeltp.wv.similarity('Playgrounds', 'Kids Activities')

0.5337816

## Treinamento em Pipeline

Construindo vocabulário

In [17]:
tuples = tuples.drop_duplicates()
sentencesTuples = tuples.values.tolist()
len(sentencesTuples)

195778

In [3]:
#Adicionando o vocabulário geral do yelp

file_name = './yelp_categories.csv'

vocabulary = pd.read_csv(file_name)


##Utilizando apenas os tipos dos PoIs para criação das sentenças no Word2Vec
vocabulary = vocabulary[['category']]

#Adaptando para sentenças do word2vec
vocabulary = vocabulary.values.tolist()
vocabulary[0:20]

[['Hotels & Travel'],
 ['Financial Services'],
 ['Home Services'],
 ['Automotive'],
 ['Nightlife'],
 ['Education'],
 ['Event Planning & Services'],
 ['Shopping'],
 ['Local Flavor'],
 ['Restaurants'],
 ['Mass Media'],
 ['Local Services'],
 ['Health & Medical'],
 ['Food'],
 ['Religious Organizations'],
 ['Professional Services'],
 ['Public Services & Government'],
 ['Pets'],
 ['Active Life'],
 ['Arts & Entertainment']]

In [ ]:
# Configurando variáveis de controle e inspeção

cores = multiprocessing.cpu_count()   #Quantidade de cores utilizados no treinamento

t = time()  #Tempo de realização do processo

s = 22   #Quantidade de bins para leitura dos arquivos

w = 0.5

osm_table = 'bins_polygons_information'

#layers = [1, 2, 3]
layers = [1]

for l in layers:

    for n in range(17, s):
        print('Trainning model (tuple of bin):', n, 'layer:', l)

        #try:

        #Lendo arquivos
        file_name = './Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(w)+'-p.parquet'
        #file_name = './Austin/w05/ITDL Partial Bins Single Layers/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(w)+'-l'+str(l)+'-p.parquet'
        #file_name = './Austin/w05/ITDL Osm Concat Bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(w)+'-'+str(osm_table)+'.parquet'

        #tuples = pd.read_csv(file_name, error_bad_lines=False)
        print("loading file:", file_name)
        tuples = pd.read_parquet(file_name)

        #Removendo linhas danificadas
        tuples = tuples.dropna()

        ##Utilizando apenas os tipos dos PoIs para criação das sentenças no Word2Vec
        tuples = tuples[['center_poi', 'context_poi']]
        #tuples = tuples[['center_poi', 'context_osm']]
        #tuples.head()

        #Adaptando para sentenças do word2vec
        sentencesTuples = tuples.values.tolist()

        sentencesTuples.extend(vocabulary)             #Complementando o vocabulário

        #Criando estrutura do skip-gram
        p2v_modeltp = Word2Vec(min_count=1,
                                window=1,
                                sg=1, #Skip-gram
                                size=70,
                                sample=6e-5, 
                                alpha=0.03, 
                                min_alpha=0.0007, 
                                negative=20,
                                workers=cores-1)


        #Criando vocubulário
        p2v_modeltp.build_vocab(sentencesTuples, progress_per=10000)


        #Treinando o modelo
        p2v_modeltp.train(sentencesTuples, total_examples=p2v_modeltp.corpus_count, epochs=30, report_delay=1)

        #Salvando em arquivo
        model_name = './Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(w)+'-p.model'
        #model_name = './Austin/w05/ITDL Partial Bins Single Layers/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(w)+'-l'+str(l)+'-p.model'
        #model_name = './Austin/w05/ITDL Osm Concat Bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(w)+'-'+str(osm_table)+'.model'

        print('saving file:', model_name)
        p2v_modeltp.save(model_name)


        #except Exception as e:
            #print(str(e))
        
print('Process finish.')

Trainning model (tuple of bin): 17 layer: 0
loading file: ./Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-17bin-wgt0.5-p.parquet
saving file: ./Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-17bin-wgt0.5-p.model
Trainning model (tuple of bin): 18 layer: 0
loading file: ./Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-18bin-wgt0.5-p.parquet
saving file: ./Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-18bin-wgt0.5-p.model
Trainning model (tuple of bin): 19 layer: 0
loading file: ./Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-19bin-wgt0.5-p.parquet
saving file: ./Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-19bin-wgt0.5-p.model
Trainning model (tuple of bin): 20 layer: 0
loading file: ./Austin/w05/ITDL Partial Bins/austin-sl-tuple-n-itdl-20bin-wgt0.5-p.parquet


In [ ]:
p2v_modeltp

# Sentenças

Os esperimentos serão realizados com o word2vec (skip-gram) com os seguintes parâmetros:
- window = 2 para algoritmos puro e com fator; windows = n para versão com sentenças de lugares.
- vector dimension = 70 (definido no artigo place2vec)
- alpha = 0.03 (não especificado no artigo)

## Treinamento Separado

### Dados em Sentenças
Nesse modelo os dados estão no formato <vn, ..., v2, v1, v0, centro, v0, v1, v2, ..., vn>.

Os vizinhos nos lados esquerdo e direito não são os mesmos

In [4]:
#csv.field_size_limit(sys.maxsize)
#sentences = pd.read_csv('sentencestpma2.csv', engine='python')
sentences = pd.read_csv('austin-sl-sentences-n-itdl-0bin-wgt0.3-wnd5-d.csv')
print(sentences.shape)
sentences.head()

(66091, 3)


,0,1,2
0,Acai Bowls,Sentinel,Local Services
1,Acai Bowls,Sentinel,Windows Installation
2,Acai Bowls,Sentinel,Home Services
3,Acai Bowls,Sentinel,Appliances & Repair
4,Acai Bowls,Sentinel,Asian Fusion


In [ ]:
#Opcional - Teoria não comprova
supsentences = pd.concat([sentences, sentences[sentences.columns[::-1]]], ignore_index=True)
print(supsentences.shape)
supsentences.head()

In [6]:
#Calculando o tamanho da janela de sentenças
#windows_size = ((len(sentences.iloc[0])-1)/2)
windows_size = (len(sentences.iloc[0])-1)
print(windows_size)

2


In [7]:
#Opcional - Utilizado em outros casos
sentencesTokenizer = list()
for id, row in sentences.iterrows():
  sentencesTokenizer.append(row.tolist())
  #break

In [ ]:
#Opcional - Utilizado para procurar palavras em uma sentença
for l in sentencesTokenizer:
  for s in l:
    if(s == 'Gas & Service Stations'):
      print("Yes baby, thank you", s)
      #break

Configuração dos núcleos de processamento

### Treinamento Sentença
Esta versão distribui os pontos vizinhos ao longo de uma sentença com o ponto central no meio da sentença. A posição dos vizinhos é definida pela distância ao ponto central.

In [35]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

In [78]:
p2v_modelst = Word2Vec(min_count=1,
                     #window=windows_size,
                     window=windows_size,
                     size=70,
                     sg=1, #Skip-gram
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [79]:
t = time()
p2v_modelst.build_vocab(sentencesTokenizer, progress_per=10000)
#p2v_modelst.build_vocab(sentencesTokenizer, progress_per=10000, trim_rule=deleteSentinel)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.01 mins


In [80]:
t = time()
p2v_modelst.train(sentencesTokenizer, total_examples=p2v_modelst.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.08 mins


In [56]:
p2v_modelst.wv.most_similar(positive=['Sentinel'])

[('Tours', 0.8263634443283081),
 ('Firearm Training', 0.8196747899055481),
 ('Party Bus Rentals', 0.8186376094818115),
 ('Private Investigation', 0.8130941987037659),
 ('Hotels & Travel', 0.8121546506881714),
 ('Automotive', 0.8108544945716858),
 ('Event Planning & Services', 0.8011888265609741),
 ('Pets', 0.7972355484962463),
 ('Wine Tours', 0.79390549659729),
 ('Transportation', 0.7902112007141113)]

In [ ]:
p2v_modelst.save('test.model')

In [ ]:
p2v_modelst.wv.most_similar(positive=['Sentinel'])

In [75]:
p2v_modelst.wv.most_similar(positive=['Pizza'])

[('Pool Halls', 0.9995749592781067),
 ('Couriers & Delivery Services', 0.9995175004005432),
 ('Farmers Market', 0.9995113015174866),
 ('Bankruptcy Law', 0.9995080828666687),
 ('Tacos', 0.9995059967041016),
 ('Eyewear & Opticians', 0.9995003938674927),
 ('Wheel & Rim Repair', 0.9994894862174988),
 ('Korean', 0.9994890689849854),
 ('Modern European', 0.9994849562644958),
 ('Home & Rental Insurance', 0.9994841814041138)]

In [73]:
p2v_modelst.wv.most_similar(positive=['Coffee & Tea'])

[('Oil Change Stations', 0.9995405077934265),
 ('Caribbean', 0.9994879961013794),
 ('Food', 0.999480664730072),
 ('Mini Golf', 0.9994805455207825),
 ('Local Flavor', 0.9994764924049377),
 ('Home Energy Auditors', 0.9994749426841736),
 ('Wheel & Rim Repair', 0.9994710087776184),
 ('Laundromat', 0.9994590878486633),
 ('Tenant and Eviction Law', 0.999457836151123),
 ('Audio/Visual Equipment Rental', 0.9994561672210693)]

In [83]:
p2v_modelst.wv.most_similar(positive=['Shopping'], topn=30)

[('Building Supplies', 0.9985255599021912),
 ('Shoe Stores', 0.9983384013175964),
 ('Bridal', 0.9980649352073669),
 ('Pizza', 0.9980148077011108),
 ('Walking Tours', 0.9980079531669617),
 ('Hardware Stores', 0.9979990720748901),
 ('Nail Technicians', 0.997900664806366),
 ('Home Energy Auditors', 0.9978245496749878),
 ('Tiling', 0.9977424144744873),
 ('Utilities', 0.9977139234542847),
 ('Hookah Bars', 0.9976836442947388),
 ('Thai', 0.9976762533187866),
 ('Kids Hair Salons', 0.9976366758346558),
 ('Masonry/Concrete', 0.9976356029510498),
 ('Modern European', 0.997622549533844),
 ('Public Transportation', 0.9975969791412354),
 ('Trailer Repair', 0.9975488781929016),
 ('Mobile Phone Repair', 0.9975332021713257),
 ('Wheel & Rim Repair', 0.9975090622901917),
 ('Animal Shelters', 0.9975043535232544),
 ('Sports Clubs', 0.9975021481513977),
 ('Decks & Railing', 0.9974951148033142),
 ('Oil Change Stations', 0.9974868893623352),
 ('Fitness/Exercise Equipment', 0.9974846243858337),
 ('Jewelry', 0.

In [ ]:
p2v_modelst.wv.similarity("Pizza", 'Bars')

0.4767316

In [ ]:
p2v_modelst.wv.similarity('Playgrounds', 'Kids Activities')

0.40262905

In [ ]:
p2v_modelst.wv.most_similar(positive=['Sentinel'])

[('Interior Design', 0.6132070422172546),
 ('Orthotics', 0.6077588200569153),
 ('Home Services', 0.599266767501831),
 ('Basketball Courts', 0.5984383821487427),
 ('Golf Lessons', 0.5901661515235901),
 ('Airport Shuttles', 0.5899162888526917),
 ('Private Tutors', 0.5883011817932129),
 ('Professional Services', 0.5881460905075073),
 ('Golf', 0.5878017544746399),
 ('Test Preparation', 0.5821858048439026)]

## Treinamento em Pipeline

Orientado a bin

In [3]:
#Adicionando o vocabulário geral do yelp

file_name = './yelp_categories.csv'

vocabulary = pd.read_csv(file_name)


##Utilizando apenas os tipos dos PoIs para criação das sentenças no Word2Vec
vocabulary = vocabulary[['category']]

#Adaptando para sentenças do word2vec
vocabulary = vocabulary.values.tolist()
vocabulary[0:20]

[['Hotels & Travel'],
 ['Financial Services'],
 ['Home Services'],
 ['Automotive'],
 ['Nightlife'],
 ['Education'],
 ['Event Planning & Services'],
 ['Shopping'],
 ['Local Flavor'],
 ['Restaurants'],
 ['Mass Media'],
 ['Local Services'],
 ['Health & Medical'],
 ['Food'],
 ['Religious Organizations'],
 ['Professional Services'],
 ['Public Services & Government'],
 ['Pets'],
 ['Active Life'],
 ['Arts & Entertainment']]

In [5]:
# Configurando variáveis de controle e inspeção

cores = multiprocessing.cpu_count()   #Quantidade de cores utilizados no treinamento

t = time()  #Tempo de realização do processo

s = 6   #Quantidade de bins para leitura dos arquivos

w = 5  #Tamanho da janela

wgt = 0.7

for n in range(19, s):
    print('Trainning model (sentence of bin):', n, '...')
    
    try:
        
        #Lendo arquivos
        
        #file_name = 'yelpdecatur-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '.csv' 
        file_name = './Austin/w05/Sentences ITDL Partial Bins Single Layers/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(w)+'-'+str(osm_table)+'.parquet'

        sentences = pd.read_csv(file_name)

        #Removendo linhas danificadas
        sentences = sentences.dropna()

        #Calculando o tamanho da janela de sentenças
        #windows_size = len(sentences.iloc[0])
        #windows_size = ((len(sentences.iloc[0])-1)/2)
        windows_size = (len(sentences.iloc[0])-1)
        print("window size:", windows_size, " sentence size:", w)
        #Transformando dados para o formato do word2vec
        sentencesTokenizer = list()
        for id, row in sentences.iterrows():
          sentencesTokenizer.append(row.tolist())


        #Criando estrutura do skip-gram
        p2v_modelst = Word2Vec(min_count=1,
                     window=windows_size,
                     size=70,
                     sg=1, #Skip-gram
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)
        
        #Criando vocubulário
        p2v_modelst.build_vocab(sentencesTokenizer, progress_per=10000)
        #p2v_modelst.build_vocab(sentencesTokenizer, progress_per=10000, trim_rule=deleteSentinel)

        #Treinando o modelo
        p2v_modelst.train(sentencesTokenizer, total_examples=p2v_modelst.corpus_count, epochs=30, report_delay=1)

        #Removendo a sentinela
        #p2v_modelst.wv.vocab.pop('Sentinel')

        #Salvando em arquivo
        model_name = 'yelpdecatur-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '.model'
        #model_name = 'yelpcambridge-sl-sentences-n-h-wfx' + str(w) + '.model'
        #model_name = 'yelpcambridge-sl-sentences-n-h-bck-wfx' + str(w) + '.model'
        #model_name = 'yelpdecatur-sl-sentences-n-itdl-'+str(n)+'bin-wg7-bnw.model'

        p2v_modelst.save(model_name)

    except Exception as e:
        print(str(e))

Orientado a Janela

In [ ]:
# Configurando variáveis de controle e inspeção

cores = multiprocessing.cpu_count()   #Quantidade de cores utilizados no treinamento

t = time()  #Tempo de realização do processo

n = 5   #Quantidade de bins para leitura dos arquivos

wnd = [5, 7, 9, 11]  #Tamanho da janela

wgt = 0.7

for w in wnd:
    print('Trainning model (sentence of bin):', n, '...')
    
    try:
        
        #Lendo arquivos
        
        file_name = 'yelpdecatur-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '.csv' 


        sentences = pd.read_csv(file_name)

        #Removendo linhas danificadas
        sentences = sentences.dropna()

        #Calculando o tamanho da janela de sentenças
        #windows_size = len(sentences.iloc[0])
        windows_size = ((len(sentences.iloc[0])-1)/2)
        print("window size:", windows_size, " sentence size:", w)
        #Transformando dados para o formato do word2vec
        sentencesTokenizer = list()
        for id, row in sentences.iterrows():
          sentencesTokenizer.append(row.tolist())


        #Criando estrutura do skip-gram
        p2v_modelst = Word2Vec(min_count=1,
                     window=windows_size,
                     size=70,
                     sg=1, #Skip-gram
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)
        
        #Criando vocubulário
        p2v_modelst.build_vocab(sentencesTokenizer, progress_per=10000)
        p2v_modelst.build_vocab(sentencesTokenizer, progress_per=10000, trim_rule=deleteSentinel)

        #Treinando o modelo
        p2v_modelst.train(sentencesTokenizer, total_examples=p2v_modelst.corpus_count, epochs=30, report_delay=1)

        #Removendo a sentinela
        #p2v_modelst.wv.vocab.pop('Sentinel')

        #Salvando em arquivo
        model_name = 'yelpdecatur-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '.model'
        #model_name = 'yelpcambridge-sl-sentences-n-h-wfx' + str(w) + '.model'
        #model_name = 'yelpcambridge-sl-sentences-n-h-bck-wfx' + str(w) + '.model'
        #model_name = 'yelpdecatur-sl-sentences-n-itdl-'+str(n)+'bin-wg7-bnw.model'

        p2v_modelst.save(model_name)

    except Exception as e:
        print(str(e))

Trainning model (sentence of bin): 5 ...
window size: 2.0  sentence size: 5
Trainning model (sentence of bin): 5 ...
window size: 3.0  sentence size: 7
Trainning model (sentence of bin): 5 ...
window size: 4.0  sentence size: 9
Trainning model (sentence of bin): 5 ...
window size: 5.0  sentence size: 11


Orientado a bin e janela

In [ ]:
print('OK')

OK


In [ ]:
# Configurando variáveis de controle e inspeção

cores = multiprocessing.cpu_count()   #Quantidade de cores utilizados no treinamento

t = time()  #Tempo de realização do processo

s = 27   #Quantidade de bins para leitura dos arquivos

wgt = 0.5

#layers = [1, 2, 3]
layers = [2]

for l in layers:


    for n in range(0, s):

      #wnd = [9, 11]  #Tamanho da janela
      wnd = [5, 7]  #Tamanho da janela
      #wnd = [7]


      for w in wnd:
          print('Trainning model (sentence of bin):', n, ' window:', w)

          try:

              #Lendo arquivos

              #file_name = './Austin/w07/Sentences distance/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-d.csv'
              file_name = './Austin/w05/Sentences ITDL Partial Bins Single Layers/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-l'+str(l)+'.parquet' 
              #file_name = './Austin/w05/Sentences ITDL Partial Bins/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-p.parquet'

              #file_name = 'longwood-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-context-wnd' + str(w) + '.csv'
              #file_name = 'longwood-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-context-r-wnd' + str(w) + '.csv'
              #file_name = 'yelpcambridge-sl-sentences-n-h-wfx' + str(w) + '.csv'
              #file_name = 'yelpcambridge-sl-sentences-n-h-bck-wfx' + str(w) + '.csv'
              #file_name = 'yelpdecatur-sl-sentences-n-itdl-'+str(n)+'bin-wg7-bnw.csv'

              sentences = pd.read_parquet(file_name)

              #Removendo linhas danificadas
              sentences = sentences.dropna()

              #Calculando o tamanho da janela de sentenças
              #windows_size = len(sentences.iloc[0])

              #Para versão de sentenças à direita e esquerda
              #windows_size = ((len(sentences.iloc[0])-1)/2)

              #Para versão de sentenças só à direita
              windows_size = (len(sentences.iloc[0])-1)

              print("window size:", windows_size, " sentence size:", w)
              #Transformando dados para o formato do word2vec
              sentencesTokenizer = list()
              for id, row in sentences.iterrows():
                sentencesTokenizer.append(row.tolist())
               
              #inserindo todas as categorias no vocabulário 
              sentencesTokenizer.extend(vocabulary)
              
                
              #Criando estrutura do skip-gram
              p2v_modelst = Word2Vec(min_count=1,
                          window=windows_size,
                          size=70,
                          sg=1, #Skip-gram
                          sample=6e-5, 
                          alpha=0.03, 
                          min_alpha=0.0007, 
                          negative=20,
                          workers=cores-1)

              #Criando vocubulário
              p2v_modelst.build_vocab(sentencesTokenizer, progress_per=10000) #Com Sentinela
              #p2v_modelst.build_vocab(sentencesTokenizer, progress_per=10000, trim_rule=deleteSentinel) # Sem sentinela

              #Treinando o modelo
              p2v_modelst.train(sentencesTokenizer, 
                                total_examples=p2v_modelst.corpus_count, 
                                epochs=30, 
                                report_delay=1)

              #Removendo a sentinela
              #p2v_modelst.wv.vocab.pop('Sentinel')

              #Salvando em arquivo

              #model_name = './Austin/w07/Sentences distance/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-d.model'
              #model_name = './Austin/w05/Sentences distance/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-d-s.model'
              model_name = './Austin/w05/Sentences ITDL Partial Bins Single Layers/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-l' +str(l)+ '-p.model'
              #model_name = './Austin/w05/Sentences ITDL Partial Bins/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-p.model'
              #model_name = './Austin-sb-02/w05/Sentences distance/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-l' +str(l)+ '-d-s.model'

              #model_name = './Austin-sb/w05/Sentences distance/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '.model'
              #model_name = './Austin-sb/w05/Sentences distance/austin-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-l'+str(l)+'.model'

              #model_name = 'longwood-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-wnd' + str(w) + '-d.model'
              #model_name = 'longwood-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-context-wnd' + str(w) + '.model'
              #model_name = 'longwood-sl-sentences-n-itdl-'+ str(n) + 'bin-wgt'+str(wgt)+'-context-rd-wnd' + str(w) + '.model'
              #model_name = 'yelpcambridge-sl-sentences-n-h-wfx' + str(w) + '.model'
              #model_name = 'yelpcambridge-sl-sentences-n-h-bck-wfx' + str(w) + '.model'
              #model_name = 'yelpdecatur-sl-sentences-n-itdl-'+str(n)+'bin-wg7-bnw.model'

              p2v_modelst.save(model_name)

          except Exception as e:
              print(str(e))
                
print('process finished.')

Trainning model (sentence of bin): 0  window: 5
window size: 2  sentence size: 5
Trainning model (sentence of bin): 0  window: 7
window size: 3  sentence size: 7
Trainning model (sentence of bin): 1  window: 5
window size: 2  sentence size: 5
Trainning model (sentence of bin): 1  window: 7
window size: 3  sentence size: 7
Trainning model (sentence of bin): 2  window: 5
window size: 2  sentence size: 5
Trainning model (sentence of bin): 2  window: 7
window size: 3  sentence size: 7
Trainning model (sentence of bin): 3  window: 5
window size: 2  sentence size: 5
Trainning model (sentence of bin): 3  window: 7
window size: 3  sentence size: 7
Trainning model (sentence of bin): 4  window: 5
window size: 2  sentence size: 5
Trainning model (sentence of bin): 4  window: 7
window size: 3  sentence size: 7
Trainning model (sentence of bin): 5  window: 5
window size: 2  sentence size: 5
Trainning model (sentence of bin): 5  window: 7
window size: 3  sentence size: 7
Trainning model (sentence of

# Teste de Embedings

In [ ]:
#Parameters
n = 0
wgt = 0.5
wnd = 5
osm_table = 'bins_polygons_information'


#model_name = './Austin/w05/ITDL Osm Concat Bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(w)+'-'+str(osm_table)+'.model'
model_name = './Austin/w05/ITDL Osm Concat Bins/austin-sl-tuple-n-itdl-0bin-wgt0.5-bins_polygons_information.model'
model = Word2Vec.load(model_name)

In [51]:
model.wv.similarity('Playgrounds', 'Parks')

0.9125352

In [50]:
model.wv.most_similar(positive=['Playgrounds'], topn=100)

[('Parks', 0.9125351905822754),
 ('Day Camps', 0.8820595741271973),
 ('Kids Activities', 0.8667660355567932),
 ('Tai Chi', 0.8627614378929138),
 ('Botanical Gardens', 0.861686646938324),
 ('Prosthetics', 0.8605790138244629),
 ('Medical Transportation', 0.8594191670417786),
 ('Pet Waste Removal', 0.8583695888519287),
 ('Skating Rinks', 0.8583691120147705),
 ('Hydro-jetting', 0.85792475938797),
 ('Swimming Pools', 0.8556530475616455),
 ('Baseball Fields', 0.8502784371376038),
 ('Active Life', 0.8501186370849609),
 ('Blood & Plasma Donation Centers', 0.8494305610656738),
 ('Tennis', 0.8491591811180115),
 ('Beaches', 0.8480761647224426),
 ('Home Staging', 0.8442037105560303),
 ('Boot Camps', 0.8434141874313354),
 ('Chicken Wings', 0.842438817024231),
 ('Pool & Billiards', 0.8389462828636169),
 ('Smokehouse', 0.8385064005851746),
 ('Hiking', 0.8384810090065002),
 ('Calligraphy', 0.8344879746437073),
 ('Christmas Trees', 0.8333354592323303),
 ('Private Schools', 0.8329429030418396),
 ('Game 

In [79]:
sentences = [['Shopping', 'amenity_parking'],
 ['Shopping', 'amenity_parking'],
 ['Shopping', 'building_commercial'],
 ['Shopping', 'building_commercial'],
 ['Shopping', 'amenity_parking'],
 ['Shopping', 'amenity_parking'],
 ['Shopping', 'building_commercial'],
 ['Shopping', 'amenity_parking'],
 ['Shopping', 'amenity_parking'],
 ['Jewelry Repair', 'amenity_parking'],
 ['Jewelry Repair', 'amenity_parking'],
 ['Jewelry Repair', 'building_commercial'],
 ['Jewelry Repair', 'building_commercial'],
 ['Jewelry Repair', 'amenity_parking'],
 ['Jewelry Repair', 'amenity_parking'],
 ['Jewelry Repair', 'building_commercial'],
 ['Jewelry Repair', 'amenity_parking'],
 ['Jewelry Repair', 'amenity_parking'],
 ['Appraisal Services', 'amenity_parking'],
 ['Appraisal Services', 'amenity_parking'],
 ['Appraisal Services', 'building_commercial'],
 ['Appraisal Services', 'building_commercial'],
 ['Appraisal Services', 'amenity_parking'],
 ['Appraisal Services', 'amenity_parking'],
 ['Appraisal Services', 'building_commercial'],
 ['Appraisal Services', 'amenity_parking'],
 ['Appraisal Services', 'amenity_parking'],
 ['Local Services', 'amenity_parking'],
 ['Local Services', 'amenity_parking'],
 ['Local Services', 'building_commercial']]
model = Word2Vec(sentences, min_count=1, window=1, size=70, sg=1)

In [80]:
model.wv.most_similar(positive=['Shopping'])

[('Appraisal Services', 0.12836101651191711),
 ('Jewelry Repair', 0.10327723622322083),
 ('Local Services', 0.06813202798366547),
 ('building_commercial', 0.0017136335372924805),
 ('amenity_parking', -0.15231953561306)]

In [81]:
model.predict_output_word(['Local Services'], topn=10)

[('amenity_parking', 0.16666926),
 ('Appraisal Services', 0.16666739),
 ('building_commercial', 0.16666718),
 ('Local Services', 0.16666552),
 ('Jewelry Repair', 0.16666542),
 ('Shopping', 0.1666652)]